<a href="https://colab.research.google.com/github/jesst0x/image-segmentation-in-dark/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'AI/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [2]:
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from torchvision.transforms import v2
import numpy as np

#Data Augmentation

In [ ]:
sample_img = Image.open('/content/drive/My Drive/AI/sample.jpg')
plt.imshow(sample_img)